<h1>Kmeans Clustering</h1>
<img src="../data/K_means.gif" width="1000" align="center">
With our knowledge of Python and now Numpy lets create an implementation of a famous machine learning algorithm "K-Means Clustering". The job of a clustering algorithm is to break a dataset into some number of "clusters" (groups), the number of clusters usually defined by the user. K-Means clustering working by iteratively updating  a pre-defined number of cluster centers. It does this by finding the distance between each datapoint and every cluster center. Datapoints are then assigned to the cluster center they are closest to and each cluster center is updated to be the mean of the new cluster. These steps are updated for some number of steps or until the cluster centers converge (they stop moving so much)<br>
<b>Lets have a look at the steps of K-means clustering</b><br>
1. Define the number of clusters "k" you want to group your data into<br>
2. Randomly initialise k vectors with the same size as each datapoint, this is the initialisation of our cluster centers<br>
3. Calculate the distance between each datapoint and each cluster center (using MSE or equivalent)<br>
4. For every datapoint find the cluster center they are closest to<br>
5. Re-calculate the cluster centers by finding the mean of every new cluster<br>
6. Repeat steps 3-6 for n steps or until convergence

In [ ]:
import matplotlib.pyplot as plt  
import numpy as np     
import torch
from load import test_x
import time
from tqdm.notebook import trange, tqdm

<b>Using the module "load" that comes with this notebook, lets load our dataset</b><br>
The dataset we'll be using is the MNIST dataset, a dataset of small, low-res handwritten digits. There are 60000 training images and 10000 test images divided up into 10 classes (digits 0-9). Here we will be using the test set (as it's a smaller set)

In [ ]:
# Sneak peak at using GPUs for computation! (Will only work if you have a cuda enabled GPU)
# device = "cpu"
gpu_indx = 0
device = torch.device(gpu_indx if torch.cuda.is_available() else "cpu")
print("cuda or cpu?:", device)

In [ ]:
# Number of datapoint
num_img = 10000  
# Number of cluster centers, 10 because the dataset contains 10 classes eg: digit 0 to 9
num_means = 10   
# We'll perform this many iterations of the algorithm
iterations = 100
# Each image is 28*28 pixels, which has been flattened to a vector 0f 784 values
data_size = 28*28
# The images are 8 bit greyscale images (values range from 0-255)
# We'll rescale the pixel values to be between 0-1 (We don't REALLY need to do this for k-means)
test_x_tensor = torch.FloatTensor((test_x.astype(float) / 255)).to(device)

<h3> Kmeans Initialization </h3>
Here we'll initialise the cluster centers to random values by creating a 10*784 matrix (2D Tensor) by randomly sampling 10 points from the dataset

In [ ]:
# Randomly generate K indicies for k datapoints from the dataset (indicies need to be int)
means = test_x_tensor[np.random.randint(0, num_img , num_means)]
eye_mat = torch.eye(num_means, device=device)

In [ ]:
# Random initial operation to initialize Pytorch
means = torch.mm(eye_mat, means)

In [ ]:
plt.figure(1, figsize=(20, 10))
img = means.cpu().float().numpy().reshape(num_means, 28, 28).transpose((1, 0, 2)).reshape(28, num_means*28)
_ = plt.imshow(img)

<h3> Kmeans Algorithm </h3>
Now implement the main steps of the K-Means clustering algorithm! Try and make it as efficient as possible and minimise the time/iteration

In [ ]:
start_time = time.time()
with torch.no_grad():
    for i in trange(iterations):
        # Add on a dimension in the right place and use broadcasting to find the differences
        diff_from_means = means.unsqueeze(0) - test_x_tensor.unsqueeze(1)

        # Using absolute sum of differences here
        dist_to_means = diff_from_means.pow(2).mean(2)

        # Expand dims is anther way to add a dimension
        indx_of_means = dist_to_means.argmin(1)

        # Create a one hot coded vector per datapoint
        a = eye_mat[indx_of_means].t()
        # Multiply to get the sums of each cluster then divide by elements per cluster to get means
        means = torch.mm(a, test_x_tensor) / a.sum(1, keepdims=True)

end_time = time.time()
print("%d iterations took %.2f seconds, which corresponds to %.4fs/iteration" % (iterations, end_time - start_time, (end_time - start_time)/iterations))

<h3>Lets visualise the the cluster centers!</h3>

In [ ]:
plt.figure(1, figsize=(20, 10))
img = means.cpu().float().numpy().reshape(num_means, 28, 28).transpose((1, 0, 2)).reshape(28, num_means*28)
_ = plt.imshow(img)